In [1]:
import xarray as xr
import cftime
import numpy as np

fp = "/beegfs/CMIP6/jdpaul3/cmip6_regrid_for_rasdaman/cmip6_regrid_mon_ensemble.nc"

In [2]:
ds = xr.open_dataset(fp, decode_cf=True)
ds

<xarray.Dataset> Size: 126GB
Dimensions:      (model: 14, scenario: 5, time: 1812, lat: 43, lon: 288)
Coordinates:
  * model        (model) int64 112B 0 1 2 3 4 5 6 7 8 9 10 11 12 13
  * scenario     (scenario) int64 40B 0 1 2 3 4
  * lat          (lat) float64 344B 90.0 89.06 88.12 87.17 ... 52.3 51.36 50.42
  * lon          (lon) float64 2kB -180.0 -178.8 -177.5 ... 176.2 177.5 178.8
  * time         (time) object 14kB 1950-01-15 12:00:00 ... 2100-12-15 12:00:00
Data variables: (12/19)
    clt          (model, scenario, time, lat, lon) float32 6GB ...
    evspsbl      (model, scenario, time, lat, lon) float32 6GB ...
    hfls         (model, scenario, time, lat, lon) float32 6GB ...
    hfss         (model, scenario, time, lat, lon) float32 6GB ...
    pr           (model, scenario, time, lat, lon) float64 13GB ...
    prsn         (model, scenario, time, lat, lon) float64 13GB ...
    ...           ...
    tasmax       (model, scenario, time, lat, lon) float32 6GB ...
    tasmin       (model, scenario, time, lat, lon) float32 6GB ...
    ts           (model, scenario, time, lat, lon) float32 6GB ...
    uas          (model, scenario, time, lat, lon) float32 6GB ...
    vas          (model, scenario, time, lat, lon) float32 6GB ...
    spatial_ref  int64 8B ...
Attributes:
    Conventions:  CF-1.8
    title:        CMIP6 Monthly Data on a Common Grid with Multi-Model Ensemb...
    description:  Monthly data from 13 CMIP6 models on a common grid, includi...
    institution:  Scenarios Network for Alaska and Arctic Planning, Universit...
    source:       CMIP6 model output
    contact:      uaf-snap-data-tools@alaska.edu
    url:          https://uaf-snap.org/

In [3]:
# test each variable to make sure the ensemble mean is calculated correctly

# historical data

lat = 65
lon = -142
time = "2000-01-15"


scenario = 0
models = slice(0, 13)  # first 13 models


for var in list(ds.data_vars):
    if var != "spatial_ref":
        print(f"Variable: {var}")

        print("Original Dataset Values:")
        values = (
            ds[var]
            .sel(
                lat=lat,
                lon=lon,
                method="nearest",
            )
            .sel(time=time, scenario=scenario, model=models)
            .values
        )

        print(values)
        print("\n Calculated Ensemble Mean:")
        print(np.nanmean(values))

        print("Dataset Ensemble Mean:")
        print(
            ds[var]
            .sel(
                lat=lat,
                lon=lon,
                method="nearest",
            )
            .sel(time=time, scenario=scenario, model=13)
            .values
        )

        print("\n\n")

Variable: clt
Original Dataset Values:
[[83.66987 ]
 [83.589226]
 [92.98942 ]
 [86.52892 ]
 [90.112686]
 [92.5549  ]
 [96.03158 ]
 [97.261894]
 [69.13762 ]
 [62.9286  ]
 [84.60915 ]
 [73.20425 ]
 [88.03362 ]
 [84.66552 ]]

 Calculated Ensemble Mean:
84.66552
Dataset Ensemble Mean:
[84.66552]



Variable: evspsbl
Original Dataset Values:
[[-8.0136465e-07]
 [-5.1908472e-07]
 [-7.8565273e-07]
 [-2.4567325e-07]
 [ 1.7288687e-06]
 [-2.0015928e-07]
 [-2.4867447e-07]
 [ 6.4609040e-08]
 [ 4.4354474e-07]
 [-4.4103717e-07]
 [ 4.4930991e-07]
 [-1.6630395e-07]
 [-4.7408412e-07]
 [-9.1977078e-08]]

 Calculated Ensemble Mean:
-9.1977086e-08
Dataset Ensemble Mean:
[-9.197708e-08]



Variable: hfls
Original Dataset Values:
[[-2.0768504 ]
 [-1.4162908 ]
 [-1.9648087 ]
 [-0.666201  ]
 [ 4.322172  ]
 [-0.56746614]
 [-0.7049919 ]
 [ 0.18316348]
 [ 1.1088618 ]
 [-1.2336146 ]
 [ 1.2762977 ]
 [-0.4220986 ]
 [-1.2313712 ]
 [-0.26101524]]

 Calculated Ensemble Mean:
-0.26101524
Dataset Ensemble Mean:
[-0.26101

[[207.60397]
 [206.263  ]
 [209.36342]
 [194.0858 ]
 [229.04094]
 [188.63182]
 [211.74406]
 [229.78119]
 [215.70398]
 [195.02763]
 [230.63168]
 [171.10318]
 [212.26584]
 [207.78818]]

 Calculated Ensemble Mean:
207.7882
Dataset Ensemble Mean:
[207.78818]



Variable: rsds
Original Dataset Values:
[[6.5044904]
 [7.40584  ]
 [5.0960717]
 [7.0660386]
 [4.832105 ]
 [5.5294013]
 [5.789456 ]
 [5.351539 ]
 [6.097329 ]
 [7.2507415]
 [5.0185037]
 [6.901528 ]
 [6.76303  ]
 [6.1235447]]

 Calculated Ensemble Mean:
6.1235437
Dataset Ensemble Mean:
[6.1235447]



Variable: sfcWind
Original Dataset Values:
[[2.6164029]
 [2.8076239]
 [3.1452255]
 [2.8011045]
 [3.393409 ]
 [1.5539917]
 [1.223113 ]
 [1.8109397]
 [2.955665 ]
 [1.9692347]
 [2.0065773]
 [2.5206642]
 [2.7176874]
 [2.4247415]]

 Calculated Ensemble Mean:
2.4247415
Dataset Ensemble Mean:
[2.4247415]



Variable: siconc
Original Dataset Values:
[[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]]


/tmp/ipykernel_939151/2310239202.py:32: RuntimeWarning: Mean of empty slice
  print(np.nanmean(values))


In [5]:
# test each variable to make sure the ensemble mean is calculated correctly

# projected data

lat = 65
lon = -142
time = "2050-01-15"


scenario = 1
models = slice(0, 13)  # first 13 models


for var in list(ds.data_vars):
    if var != "spatial_ref":
        print(f"Variable: {var}")

        print("Original Dataset Values:")
        values = (
            ds[var]
            .sel(
                lat=lat,
                lon=lon,
                method="nearest",
            )
            .sel(time=time, scenario=scenario, model=models)
            .values
        )

        print(values)
        print("\n Calculated Ensemble Mean:")
        print(np.nanmean(values))

        print("Dataset Ensemble Mean:")
        print(
            ds[var]
            .sel(
                lat=lat,
                lon=lon,
                method="nearest",
            )
            .sel(time=time, scenario=scenario, model=13)
            .values
        )

        print("\n\n")

Variable: clt
Original Dataset Values:
[[89.471695]
 [80.04079 ]
 [      nan]
 [86.26216 ]
 [91.953094]
 [92.65354 ]
 [91.517876]
 [94.489456]
 [62.117462]
 [62.93853 ]
 [80.25916 ]
 [81.75568 ]
 [92.62666 ]
 [83.84051 ]]

 Calculated Ensemble Mean:
83.84051
Dataset Ensemble Mean:
[83.84051]



Variable: evspsbl
Original Dataset Values:
[[-1.4496624e-07]
 [-4.6953272e-07]
 [           nan]
 [-3.4780524e-07]
 [ 1.8423696e-06]
 [ 1.0234568e-07]
 [-1.9948602e-07]
 [-8.6433111e-08]
 [-3.3505276e-07]
 [ 6.8707163e-07]
 [ 5.9280660e-07]
 [-6.0456387e-07]
 [-3.7781973e-07]
 [ 5.4911144e-08]]

 Calculated Ensemble Mean:
5.4911155e-08
Dataset Ensemble Mean:
[5.4911144e-08]



Variable: hfls
Original Dataset Values:
[[-0.35044044]
 [-1.2797098 ]
 [        nan]
 [-0.928408  ]
 [ 4.6059237 ]
 [ 0.29015887]
 [-0.5655577 ]
 [-0.24505429]
 [-0.8376319 ]
 [ 1.927391  ]
 [ 1.6839334 ]
 [-1.6325666 ]
 [-0.97227407]
 [ 0.14131366]]

 Calculated Ensemble Mean:
0.14131366
Dataset Ensemble Mean:
[0.14131366

/tmp/ipykernel_939151/3768759034.py:32: RuntimeWarning: Mean of empty slice
  print(np.nanmean(values))
